In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
def miss_values(df):
    for column in df:
        # Test whether column has null value
        if len(df[column].apply(pd.isnull).value_counts()) > 1:
            print(column+" has missing value")
            #if column is numeric, fill null with mean
            if df[column].dtype in ('int64','float64'):
                df[column] = df[column].fillna(df[column].mean())
            else:
                df[column] = df[column].fillna("unknown")

In [ ]:
def factor_encoding(df):
    for column in df:
        if df[column].dtype == 'object':
            df = df.merge(pd.get_dummies(data=df[column],prefix=column),right_index=True,left_index=True)
            del df[column]
    return df

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
train['source'] = 1.0
test['source'] = 0.0
alls = pd.concat([train,test],ignore_index=True)
miss_values(alls)
alls = factor_encoding(alls)

In [ ]:
train = alls[alls['source'] == 1.0]
train.drop(['source'],inplace=True,axis=1)
test = alls[alls['source'] == 0.0]
test.drop(['source'],inplace=True,axis=1)
test.drop(['SalePrice'],inplace=True,axis=1)

In [ ]:
x = train.drop(['SalePrice'],axis=1)
Y = train['SalePrice']

In [ ]:
x_train, x_vali, Y_train, Y_vali = train_test_split(x, Y, test_size=0.25, random_state=42)
#Tranform dataframe into array
x_train_array = pd.DataFrame.as_matrix(x_train)
x_vali_array = pd.DataFrame.as_matrix(x_vali)
Y_train_array = Y_train.values
Y_vali_array = Y_vali.values
dtrain = xgb.DMatrix(x_train_array, label=Y_train_array)
dvali = xgb.DMatrix(x_vali_array, label=Y_vali_array)
x_array = pd.DataFrame.as_matrix(x)
Y_array = Y.values
dt = xgb.DMatrix(x_array, label=Y_array)
dtest_array = xgb.DMatrix(pd.DataFrame.as_matrix(test))

In [ ]:
#Setting XGBoosting Parameters
param = {'max_depth':3, 'eta':1, 'silent':0, 'eval_metric':'rmse'}
evallist  = [(dvali,'eval'), (dtrain,'train')]
num_round = 30
bst = xgb.train(param,dt,num_round,evallist)

In [ ]:
bst.save_model('first_iter.model')
ypred = bst.predict(dtest_array)

In [ ]:
xgb.plot_tree(bst, num_trees=2)

In [ ]:
test['SalePrice'] = list(ypred)

In [ ]:
submission = test[['Id','SalePrice']]

In [ ]:
submission.to_csv('submission.csv',index=False)